# CLUSTERING WITH DBSCAN

Cluster Analysis is an important problem in data analysis. Data scientists use clustering to identify malfunctioning servers, group genes with similar expression patterns, or various other applications.

K-Means determines `k` centroids in the data and clusters points by assigning them to the nearest centroid. While K-Means is easy to understand and implement in practice, the algorithm has no notion of outliers, so all points are assigned to a cluster even if they do not belong in any. In the domain of anomaly detection, this causes problems as anomalous points will be assigned to the same cluster as "normal" data points. The anomalous points pull the cluster centroid towards them, making it harder to classify them as anomalous points.

Compared to centroid-based clustering like K-Means, density-based clustering works by identifying "dense" clusters of points, allowing it to learn clusters of arbitrary shape and identify outliers in the data. 

## Preliminary: ɛ-Balls and neighborhood density

Before we can discuss density-based clustering, we first need to cover a topic that you may have seen in a topology course: ɛ-neighborhoods.

The general idea behind ɛ-neighborhoods is given a data point, we want to be able to discover the data points in the space around it. Formally, for some real-valued ɛ > 0 and some point p, the ɛ-neighborhood of p is defined as the set of points that are at most distance ɛ away from p.

If you think back to geometry, the shape in which all points are equidistant from the center is the circle. In 2D space, the ɛ-neighborhood of a point p is the set of points contained in a circle of radius ɛ, centered at p. In 3D space, the ɛ-neighborhood is a sphere of radius ɛ, centered at p, and in higher dimensional space, the ɛ-neighborhood is just the N-sphere of radius ɛ, centered at p.

Let's consider an example to make this idea more concrete - scattered 100 data points in the interval [1,3] X [2,4]. Let's pick the point (3,2) to be our point p.

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Epsilon-Balls

In [ ]:
test_pt = np.array([3,2])
datapoints = 1.5 * (np.random.rand(100,2)-0.5) + test_pt

In [ ]:
def compute_balls(data, epsilon = 0.5):
    circle = np.linspace(0, 2*np.pi, 100)
    neighborhood = []
    data_x, data_y = data
    #Create a circle of radius epsilon
    for direction in circle:
        x_pos = data_x + epsilon*np.cos(direction)
        y_pos = data_y + epsilon*np.sin(direction)
        neighborhood.append((x_pos,y_pos))
    return neighborhood

In [ ]:
neighborhood = compute_balls(test_pt, epsilon = 0.15)

In [ ]:
X,Y = zip(*neighborhood)
X,Y = np.array(X),np.array(Y)

In [ ]:
count = len(list(filter(lambda x: x < 0.15, cdist(datapoints, test_pt.reshape(1,2)))))

In [ ]:
plt.fill_between(X, Y, alpha = 0.25, color = 'g', label = "Neighborhood")
plt.scatter(datapoints[:,0], datapoints[:,1], s = 30, color = 'r',label = "Datapoints")
plt.scatter(test_pt[0],test_pt[1], s = 35, label = "Point p")
plt.xlim(2,4)
plt.ylim(1,3)
plt.legend(loc = 2)
plt.title("Density-Reachable with radius 0.15")
#plt.savefig("results/neighborhood_reachable_base.png", format="PNG")

The opaque green oval represents our neighborhood, and there are 6 data points in this neighborhood. Since we scattered 100 data points and 6 are in the neighborhood, this means that a 6% of the data points are contained within the neighborhood of p with radius 0.15.

Now that we have defined what we mean by a "neighborhood", we'll introduce the next important concept: the notion of a "density" for a neighborhood (we're building up to describing "density-based clustering, after all).

In a grade-school science class, children are taught that density = mass/volume. Let's use this idea of mass divided by volume to define density at some point p. If we consider some point p and its neighborhood of radius ɛ, we can define the mass of the neighborhood as the number of data points (or alternatively, the fraction of data points) contained within the neighborhood, and the volume of the neighborhood is volume of the resulting shape of the neighborhood. In the 2D case, the neighborhood is a circle, so the volume of the neighborhood is just the area of the resulting circle. In the 3D and higher dimensional case, the neighborhood is a sphere or n-sphere, so we can calculate the volume of this shape.

The mass is the number of data points in the neighborhood, so mass = 31. The volume is the area of the circle, so volume = π0.52 = π/4. Therefore, our local density approximation at *p = (3,2) is calculated as density = mass/volume = 31/(π/4) = 124/π ~= 39.5.

This value is meaningless by itself, but if we calculate the local density approximation for all points in our dataset, we could cluster our points by saying that points that are nearby (contained in the same neighborhood) and have similar local density approximations belong in the same cluster. If we decrease the value of ɛ, we can construct smaller neighborhoods (less volume) that would also contain fewer data points. Ideally, we want to identify highly dense neighborhoods where most of the data points are contained in these neighborhoods, but the volume of each of these neighborhoods is relatively small.

While this is not exactly what DBSCAN algorithm does, it forms the general intuition behind density-based clustering.

To recap, we discussed the ɛ-neighborhoods and how they allow us to reason about the space around a particular point. Then we defined a notion of density at a particular point for a particular neighborhood. In the next section, we'll discuss the DBSCAN algorithm where the ɛ-ball is a fundamental tool for defining clusters.

In [ ]:
neighborhood = compute_balls(test_pt, epsilon = 0.50)
X,Y = zip(*neighborhood)
X,Y = np.array(X),np.array(Y)
count = len(list(filter(lambda x: x < 0.15, cdist(datapoints, test_pt.reshape(1,2)))))

plt.fill_between(X, Y, alpha = 0.55, color = 'g', label = "Neighborhood")
plt.scatter(datapoints[:,0], datapoints[:,1], s = 30, color = 'r',label = "Datapoints")
plt.scatter(test_pt[0],test_pt[1], s = 35, label = "Point p")
plt.xlim(2,4)
plt.ylim(1,3)
plt.legend(loc = 2)
plt.title("Density-Reachable with radius 0.50")

## DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is the most well-known density-based clustering algorithm.

Unlike K-Means, DBSCAN does not require the number of clusters as a parameter. Rather it infers the number of clusters based on the data, and it can discover clusters of arbitrary shape (for comparison, K-Means usually discovers spherical clusters). As I said earlier, the ɛ-neighborhood is fundamental to DBSCAN to approximate local density, so the algorithm has two parameters:

1. ɛ: The radius of our neighborhoods around a data point p.
2. minPts: The minimum number of data points we want in a neighborhood to define a cluster.

Using these two parameters, DBSCAN categories the data points into three categories:

- `Core Points`: A data point p is a core point if Nbhd(p,ɛ) [ɛ-neighborhood of p] contains at least minPts ; | Nbhd(p,ɛ) | >= minPts.

- `Border Points`: A data point *q is a border point if Nbhd(q, ɛ) contains less than minPts data points, but q is reachable from some core point p.

- `Outlier`: A data point o is an outlier if it is neither a core point nor a border point. Essentially, this is the "other" class.

These definitions may seem abstract, so let's cover what each one means in more detail.

### CORE POINTS

Core Points are the foundations for DBSCAN clusters and are based on the density approximation. We use the same ɛ to compute the neighborhood for each point, so the volume of all the neighborhoods is the same. However, the number of other points in each neighborhood is what differs. Recall that we can think of the number of data points in the neighborhood as its mass. The volume of each neighborhood is constant, and the mass of neighborhood is variable, so by putting a threshold on the minimum amount of mass needed to be core point, we are essentially setting a minimum density threshold. Therefore, core points are data points that satisfy a minimum density requirement. Our clusters are built around our core points (hence the core part), so by adjusting our minPts parameter, we can fine-tune how dense our clusters cores must be.

### BORDER POINTS

Border Points are the points in our clusters that are not core points. In the definition above for border points, we used the term density-reachable. we have not defined this term yet, but the concept is simple. To explain this concept, let's revisit our neighborhood example with epsilon = 0.15. Consider the point r (the black dot) that is outside of the point p's neighborhood.

<img src="images/dbscan-1.png" alt="image1" style="width: 600px;height: 400px"/>


All the points inside the point p's neighborhood are said to be directly reachable from p. Now, let's explore the neighborhood of point q, a point directly reachable from p. The yellow circle represents q's neighborhood.


<img src="images/neighborhood_smaller.png" alt="image1" style="width: 600px;height: 400px"/>


Now while our target point r is not our starting point p's neighborhood, it is contained in the point q's neighborhood. This is the idea behind density-reachable: If we can get to the point r by jumping from neighborhood to neighborhood, starting at a point p, then the point r is density-reachable from the point p.

<img src="images/neighborhood_reachable.png" alt="image1" style="width: 600px;height: 400px"/>


As an analogy, we can think of density-reachable points as being the "friends of a friend". If the directly-reachable of a core point p are its "friends", then the density-reachable points, points in neighborhood of the "friends" of p, are the "friends of its friends". One thing that may not be clear is density-reachable points is not limited to just two adjacent neighborhood jumps. As long as you can reach the point doing "neighborhood jumps", starting at a core point p, that point is density-reachable from p, so "friends of a friend of a friend ... of a friend" are included as well.

It is important to keep in mind that this idea of density-reachable is dependent on our value of ɛ. By picking larger values of ɛ, more points become density-reachable, and by choosing smaller values of ɛ, less points become density-reachable.


## OUTLIERS

Finally, we get to our "other" class. Outliers are points that are neither core points nor are they close enough to a cluster to be density-reachable from a core point. Outliers are not assigned to any cluster and, depending on the context, may be considered anomalous points.


## APPLICATION

DBSCAN is avaible in Scikit-Learn, and because this implementation is scalable and well-tested, we will be using it to demonstrate how DBSCAN works in practice.

The steps to the DBSCAN algorithm are:

1. Pick a point at random that has not been assigned to a cluster or been designated as an `outlier`. Compute its neighborhood to determine if it's a `core point`. If yes, start a cluster around this point. If no, label the point as an `outlier`.
2. Once we find a `core point` and thus a cluster, expand the cluster by adding all `directly-reachable` points to the cluster. Perform "neighborhood jumps" to find all `density-reachable` points and add them to the cluster. If an an `outlier` is added, change that point's status from `outlier` to `border point`.
3. Repeat these two steps until all points are either assigned to a cluster or designated as an `outlier`.

In [ ]:
! head -10 ../data/wholesale.csv

In [ ]:
# DBSCAN with eps=1 and min_samples=3
# https://archive.ics.uci.edu/ml/datasets/Wholesale+customers#
from sklearn.cluster import DBSCAN
import pandas as pd

wholesale = pd.read_csv("../data/wholesale.csv")
wholesale.drop(["Channel", "Region"], axis = 1, inplace = True)

After dropping two fields that identify the customer, we can examine the first few rows of this dataset.

In [ ]:
wholesale.shape

In [ ]:
wholesale.head(5)

So we can visualize the data, we are going to use only two of these attributes:

- `Groceries`: The customer's annual spending (in some monetary unit) on grocery products.
- `Milk`: The customer's annual spending (in some monetary unit) on milk products.

In [ ]:
wholesale_selected = wholesale[["Grocery", "Milk"]]
wholesale_selected = wholesale.as_matrix().astype("float32", copy = False)

In [ ]:
wholesale_selected[:10]

Because the values of the data are in the thousands, we are going to normalize each attribute by scaling it to 0 mean and unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

stdscaler = StandardScaler().fit(wholesale_selected)
wholesale_transformed = stdscaler.transform(wholesale_selected)

Now, let's visualize the normalized dataset.

In [ ]:
plt.scatter(wholesale_transformed[:, 0], wholesale_transformed[:, 1])
plt.xlabel("Groceries")
plt.ylabel("Milk")
plt.title("Wholesale Data - Groceries and Milk")
plt.savefig("wholesale.png", format = "PNG")

As you can see, there is positive correlation between grocery purchases and milk product purchases. There is a cluster centered about the mean milk purchase (milk = 0) and the mean groceries purchase (groceries = 0). In addition, there are a handful of outliers pertaining to customers who buy more groceries or milk products compared to other customers.

With DBSCAN, we want to identify this main cluster of customers, but we also want to flag customers with more unusual annual purchasing habits as outliers.

We will construct a DBSCAN object that requires a minimum of 15 data points in a neighborhood of radius 0.5 to be considered a core point.

In [ ]:
dbsc = DBSCAN(eps = .5, min_samples = 15).fit(wholesale_transformed)

Next, we can extract our cluster labels and outliers to plot our results.

In [ ]:
labels = dbsc.labels_

# Return an array of zeros with the same shape and type as a given array.
core_samples = np.zeros_like(labels, dtype = bool)

In [ ]:
unique_labels = np.unique(labels)
unique_labels

In [ ]:
wholesale['cluster'] = labels
wholesale.sort_values('cluster')

In [ ]:
unique_labels = np.unique(labels)
colors = plt.cm.Spectral(np.linspace(0,1, len(unique_labels)))

In [ ]:
for (label, color) in list(zip(unique_labels, colors)):
    class_member_mask = (labels == label)
    xy = wholesale_transformed[class_member_mask & core_samples]
    print (len(core_samples))
    #plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor = color, markersize = 10)
    
    xy2 = wholesale_transformed[class_member_mask & ~core_samples]
    #plt.plot(xy2[:,0], xy2[:,1], 'o', markerfacecolor = color, markersize = 5)

plt.title("DBSCAN on Wholsesale data")
plt.xlabel("Grocery (scaled)")
plt.ylabel("Milk (scaled)")
plt.savefig("dbscan_wholesale.png", format = "PNG")

Lining up with our intuition, the DBSCAN algorithm was able to identify one cluster of customers. In addition, it was able to flag customers whose annual purchasing behavior deviated too heavily from other customers.

Because the outliers corresponded to customers with more extreme purchasing behavior, the wholesale distributor could specifically target these customers with exclusive discounts to encourage larger purchases.

As a baseline, let's run K-Means with two clusters on this dataset. The big blue dot represents the centroid for the black cluster, and the big gold dot represents the centroid for the white cluster.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 2).fit(wholesale_transformed)
labels_kmeans = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
plt.scatter(wholesale_transformed[:,0], wholesale_transformed[:,1], c = labels_kmeans)
plt.scatter(centroids[:,0], centroids[:,1], c = ["gold","blue"], s = 60 )
plt.savefig("kmeans_wholesale.png", format = "PNG")

While the white clusters appears to capture most of the outliers, the cluster basically captures customers who purchase relatively more goods. If we designate the white cluster as the "anomalous" cluster, then we basically flag any customer who purchases a lot of milk or groceries. If you were the wholesale distributor, then you would be calling your better customers, the ones whom you make more money from, anomalies.

## BEER EXAMPLE CONTINUED

In [1]:
# beer dataset
import pandas as pd
url = '../data/beer.txt'
beer = pd.read_csv(url, sep=' ')
X = beer.drop('name', axis=1)

# center and scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [2]:
# DBSCAN with eps=1 and min_samples=3
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=3, min_samples=3)
db.fit(X_scaled)

DBSCAN(algorithm='auto', eps=3, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=3, n_jobs=1, p=None)

In [3]:
beer_scaled = pd.DataFrame(X_scaled, columns=beer.columns[1:])

# review the cluster labels
db.labels_

array([ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int64)

In [4]:
beer['cluster'] = db.labels_
beer.sort_values('cluster')
beer.groupby('cluster').mean()

,calories,sodium,alcohol,cost
cluster,,,,
-1,157.000000,15.000000,0.900000,0.480000
0,131.263158,14.947368,4.415789,0.495263


In [5]:
beer

,name,calories,sodium,alcohol,cost,cluster
0,Budweiser,144,15,4.7,0.43,0
1,Schlitz,151,19,4.9,0.43,0
2,Lowenbrau,157,15,0.9,0.48,-1
3,Kronenbourg,170,7,5.2,0.73,0
4,Heineken,152,11,5.0,0.77,0
5,Old_Milwaukee,145,23,4.6,0.28,0
6,Augsberger,175,24,5.5,0.40,0
7,Srohs_Bohemian_Style,149,27,4.7,0.42,0
8,Miller_Lite,99,10,4.3,0.43,0
9,Budweiser_Light,113,8,3.7,0.40,0
